In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re

In [2]:
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',index_col=False)

In [3]:
org_ori_df.head(3)

,org_uuid,name,legal_name,homepage_url,country_code,state_code,region,city,address,postal_code,...,acquirer_state_code,acquirer_region,acquirer_city,acquisition_type,acquired_on,price_usd,price,price_currency_code,parent_uuid,parent_name
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,...,NY,New York,New York,acquisition,2013-12-16,30000000.0,30000000.0,USD,NaN,NaN
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,"Digg Holdings, LLC",http://www.digg.com,USA,NY,New York,New York,NaN,NaN,...,NY,New York,New York,acquisition,2012-07-12,16000000.0,16000000.0,USD,NaN,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"Facebook, Inc.",http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
org_ori_df.columns

Index(['org_uuid', 'name', 'legal_name', 'homepage_url', 'country_code',
       'state_code', 'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type

In [5]:
org_ori_df.isnull().sum(axis=0)

org_uuid                              0
name                                  0
legal_name                       121729
homepage_url                       6389
country_code                          0
state_code                        90056
region                                0
city                                  0
address                           50257
postal_code                       56895
status                                0
short_description                     2
category_list                         0
category_groups_list                  0
num_funding_rounds                    0
total_funding_usd                 43649
total_funding                     43649
total_funding_currency_code       43649
founded_on                            0
last_funding_on                       9
closed_on                        170785
employee_count                        0
email                             52692
phone                             63762
facebook_url                      76415


In [6]:
len(org_ori_df)

176985

In [7]:
org_ori_df.groupby('status')['org_uuid'].nunique()

status
acquired      19757
closed        10276
ipo            6143
operating    140809
Name: org_uuid, dtype: int64

In [8]:
jobs_df = pd.read_csv(R"d:/msc-project/data/pre-processed/jobs_preprocessed.csv",encoding='utf-8')

In [9]:
jobs_df.columns

Index(['job_uuid', 'job_name', 'people_uuid', 'person_name', 'org_uuid',
       'org_name', 'started_on', 'ended_on', 'is_current', 'title', 'job_type',
       'is_founder'],
      dtype='object')

In [10]:
num_founder = pd.DataFrame(jobs_df.groupby('org_uuid')['is_founder'].sum())

In [11]:
org_processed_df = org_ori_df[['org_uuid','name']]

In [12]:
org_processed_df=pd.merge(org_processed_df,
                         num_founder,
                         on='org_uuid',
                         how='left'
                         )

In [14]:
org_processed_df.rename(columns={'is_founder':'num_founder'},inplace=True)

In [15]:
org_processed_df

,org_uuid,name,num_founder
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,2.0
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,1.0
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,7.0
3,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,2.0
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,2.0
...,...,...,...
176980,ab278d7c-eb9f-4ed3-8c65-24b7685145a4,Frame,1.0
176981,e3baa88d-8592-40cc-8ff5-c40254b444e2,Get Harley,1.0
176982,03249b8e-ea2b-4839-a409-232f796f0e62,boomerank,0.0
176983,e2760a4a-4645-4e09-a201-3f92807123be,Switchback Systems,1.0


In [17]:
org_processed_df['num_founder'].value_counts()

1.0     45835
0.0     38934
2.0     29692
3.0     11644
4.0      3585
5.0       928
6.0       294
7.0        86
8.0        34
9.0        17
10.0        7
12.0        3
13.0        1
22.0        1
34.0        1
11.0        1
19.0        1
Name: num_founder, dtype: int64

In [67]:
org_processed_df.isnull().sum(axis=0)

org_uuid          0
name              0
is_founder    45921
dtype: int64

In [16]:
org_processed_df.dropna(subset=['num_founder'],inplace=True)

In [70]:
len(org_processed_df)

131064

In [83]:
jobs_df.columns

Index(['job_uuid', 'job_name', 'people_uuid', 'person_name', 'org_uuid',
       'org_name', 'started_on', 'ended_on', 'is_current', 'title', 'job_type',
       'is_founder'],
      dtype='object')

In [24]:
year_founded = list()

for date in list(org_ori_df.founded_on):
    year_founded.append(date.split('-')[0])

In [25]:
year_founded.uni

['2005',
 '2004',
 '2004',
 '2005',
 '2006',
 '2006',
 '2001',
 '1996',
 '2007',
 '2006',
 '2005',
 '1995',
 '2006',
 '2007',
 '1999',
 '2006',
 '2005',
 '2001',
 '1984',
 '2005',
 '2002',
 '2004',
 '2007',
 '2007',
 '2006',
 '2006',
 '2005',
 '2004',
 '2004',
 '2005',
 '2005',
 '2005',
 '2005',
 '2006',
 '2007',
 '2006',
 '2006',
 '2006',
 '2003',
 '2002',
 '2004',
 '2007',
 '2005',
 '2004',
 '2006',
 '2005',
 '2007',
 '2007',
 '2006',
 '2006',
 '2006',
 '2006',
 '2005',
 '2003',
 '2003',
 '2006',
 '2005',
 '2006',
 '1982',
 '2006',
 '2007',
 '2005',
 '2003',
 '2000',
 '2005',
 '2004',
 '2003',
 '2006',
 '1999',
 '2003',
 '2007',
 '2005',
 '2004',
 '2005',
 '2002',
 '2006',
 '2006',
 '2004',
 '2005',
 '2005',
 '2005',
 '2005',
 '1998',
 '2005',
 '2004',
 '2003',
 '1999',
 '1999',
 '2004',
 '2004',
 '2007',
 '1999',
 '2004',
 '2000',
 '2004',
 '1999',
 '2005',
 '2006',
 '2004',
 '2003',
 '2004',
 '2005',
 '2006',
 '2003',
 '2006',
 '2005',
 '2000',
 '2004',
 '2006',
 '1999',
 '2005',
 

In [86]:
#compute duration of work experience
jobs_df_not_current = jobs_df[jobs_df.is_current == False]
jobs_df_not_current.dropna(subset=['started_on','ended_on'],inplace=True)
jobs_df_not_current['duration'] = jobs_df_not_current['ended_on'] - jobs_df_not_current ['started_on']

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


TypeError: unsupported operand type(s) for -: 'str' and 'str'